# Training Material: SAR Data Processing and Change Detection for Dam Monitoring (Yamba Dam Case Study)

## I. Introduction to SAR and ALOS-2 Data (Module 8.1)

### A. What is SAR?

Synthetic Aperture Radar (SAR) is a remote sensing technology that uses radar signals to create images of the Earth's surface. Unlike optical sensors, SAR can penetrate clouds and operate day and night, providing valuable data regardless of weather or lighting conditions. SAR's ability to measure surface roughness and other properties makes it particularly useful for various applications, including dam monitoring.

### B. ALOS-2 Satellite Overview (8.1.1)

The Advanced Land Observing Satellite-2 (ALOS-2), also known as "Daichi-2," is a Japanese Earth observation satellite equipped with the PALSAR-2 L-band SAR sensor.  Its mission objectives include disaster monitoring, land use mapping, and resource exploration. PALSAR-2's L-band frequency allows for penetration of vegetation and some ground layers, making it suitable for applications like water resource management.

### C. Yamba Dam Case Study Introduction

Yamba Dam, located in Japan, serves as our case study. Monitoring its operations, including water levels and surrounding land cover, is crucial for effective water resource management and ensuring the dam's structural integrity.

## II. SAR Data Preprocessing and Visualization (Module 8.1)

### A. Data Acquisition and Access

SAR data can be acquired from various sources, including the Alaska Satellite Facility (ASF) and the Japan Aerospace Exploration Agency (JAXA).  This training uses sample ALOS-2 data downloaded from an OwnCloud instance (example shown below).

In [ ]:
!wget --content-disposition "https://owncloud.glodal-inc.net/owncloud/index.php/s/fORunC9aDo38G5s/download" && unzip Yamba.zip

### B. Data Format (8.1.2.a)

The data is provided in GeoTIFF format, a widely used format for geospatial raster data. GeoTIFF files contain georeferencing information, allowing them to be easily integrated into GIS software.

### C. Python Basics for SAR Data Handling (8.1.2.b, 8.1.2.g)

In [ ]:
!pip install rasterio matplotlib numpy

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np

with rasterio.open('/home/jovyan/shared/Arissara/ALOS-2/Yamba/Finalize/Yamba/IMG-HH-ALOS2310942880-200225-UBSR2.1GUD.tif') as src:
    image = src.read(1)

plt.figure(figsize=(8, 8))
plt.imshow(image, cmap='gray')
plt.title('SAR Image')
plt.show()

plt.hist(image.flatten(), bins=256)
plt.title('Histogram')
plt.show()

### D. Speckle Noise Reduction (8.2)

Speckle noise is a common phenomenon in SAR imagery. We will use the Lee filter to reduce this noise and improve image quality.

In [ ]:
from scipy.ndimage import uniform_filter

def lee_filter(img, size=5):
    img_mean = uniform_filter(img, (size, size))
    img_sqr_mean = uniform_filter(img**2, (size, size))
    img_variance = img_sqr_mean - img_mean**2
    overall_variance = np.var(img)
    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)
    return img_output

filtered_image = lee_filter(image)
plt.figure(figsize=(8, 8))
plt.imshow(filtered_image, cmap='gray')
plt.title('Filtered SAR Image')
plt.show()

## III. Dam Water Extent Delineation (Module 8.2)

### A. SAR Backscatter and Water Detection (8.2.1.1)

Water typically appears dark in SAR images due to its smooth surface, which reflects radar signals away from the sensor. However, other factors like terrain shadow can also appear dark, posing challenges for water detection.

### B. Otsu's Thresholding Method (8.2.1.2)

Otsu's method is used to automatically determine a threshold for separating water from non-water pixels based on the image histogram.

In [ ]:
from skimage.filters import threshold_otsu

othreshold = threshold_otsu(filtered_image)
water_mask = filtered_image < othreshold
plt.imshow(water_mask, cmap='Blues')
plt.title('Water Mask')
plt.show()

### C. Post-processing and Refinement

Post-processing steps like median filtering and vectorization can be applied to refine the water mask and calculate the water area.

In [ ]:
from scipy.ndimage import median_filter
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape

water_mask_filtered = median_filter(water_mask, size=3)

results = ({'properties': {'raster_val': v}, 'geometry': s} for i, (s, v) 
in enumerate(shapes(water_mask_filtered.astype('uint8'), transform=src.transform)))
water_gdf = gpd.GeoDataFrame.from_features(list(results), crs=src.crs)

print(f"Water Area: {water_gdf.geometry.area.sum() / 10000:.2f} hectares")

## IV. Change Detection Analysis (Module 8.3)

### A. Time-Series Data Preparation (8.3.1.b)

Change detection involves analyzing SAR data from different time points. This requires preprocessing steps like calibration, filtering, and layer stacking (demonstrated in previous modules).

### B. Visualization and Interpretation of Change (8.3.1.c, 8.3.2)

Changes in water extent can be visualized and interpreted using tools like the Temporal/Spectral Profile tool in QGIS. This allows for analyzing backscatter changes over time.

### C. (Optional) Accuracy Assessment (8.2.1.3)

Accuracy assessment can be performed using ground truth data and metrics like overall accuracy and the Kappa coefficient.

## V. Forest and Oil Palm Monitoring with ALOS-2 (Module 9)

(Optional content on forest and oil palm monitoring using similar SAR processing techniques)

## VI. Conclusion and Further Resources

This training covered essential SAR data processing and change detection techniques for dam monitoring.  Further resources and tutorials can be found online.